# Rag V2 Framework

In [ ]:
from RAG.RAG import RAG
rag = RAG(
    data_models=[
                #  {"name":"travel",
                #   "type":"db",
                #   "data": ["flights", "accommodations", "activities", "services", "insurance"],
                #   "LLM_model":"gemini-pro-ultra",
                #   "embedding_model":"models/embedding-001"},

                #   {"name":"islam",
                #    "type":"db",
                #    "data": ["hadith", "quran"],
                #    "LLM_model":"claude-3-opus-20240229",
                #    "embedding_model":"models/embedding-001"},

                   {"name": "hadith.com",
                    "type": "file",
                    "LLM_model": "claude-3-opus-20240229",
                    "embedding_model": "models/embedding-001"},
                ],
        ) 

result = rag.query(
    # query_text="plan me a travel package: i want replicate the experience of Moses travelling from Egypt to sinai, with a side quest with Al-khidr next month",
    query_text="Is there a hadith that mentions the story of Moses and Al-khidr?",
    # budget_constraint = 10000,
    # data_sources=["travel", "islam"],
    # supplementary=["AI_video_of_Moses_and_Al-khidr.mp4"],
    # focus="historical",  # Optional focus parameter: "historical", "religious", "cultural", "adventure", "educational"
    instruction_mapper = {
                            "models": [ 
                                {"type": "NLP", "model": "spaCy"},           # favor rule-based extraction for efficiency.
                                # {"type": "LLM", "model": "gemini-pro-ultra"} # rely on the LLM to interpret subtleties.
                            ] # Complex Query: The list format above would enable a sequential hybrid approach leveraging both techniques.
                         } 
)

# Example result (structured)
# print(result["travel_package"]) 
print(result)


No Hadith found containing 'Al-Khidr'.


No Hadiths found in Hisn al-Muslim section.


In [7]:
from openai import api_key, Completion  # Assuming you're using the OpenAI API
def construct_prompt(instruction, text_chunk):
    return f"Instruction: {instruction} \nDocument Text: {text_chunk} \nAnswer:"

def RagParse(parsing_instruction, result_type="markdown",):
    """
    RagParse function primarily leveraging an LLM for parsing instruction execution. 
    """
    # ... (Setup your API Key if needed) ...

    def load_data(data_path):
        try:
            with open(data_path, "r") as f:
                document_text = f.read() 
            return {"document_text": document_text}  
        except FileNotFoundError:
            print(f"Error: File not found at {data_path}")
            return {}

    def process_document(document):
        benefits_strings = []

        # (1) Split document into potential sections or chunks 
        document_chunks = split_document(document["document_text"])  

        for chunk in document_chunks:
            # (2) LLM Prompt Construction for Each Chunk
            prompt = construct_prompt(parsing_instruction, chunk) 

            # (3) Call Gemini API
            response = Completion.create(
                engine="gemini-pro-ultra-0225",  # Or your preferred engine
                prompt=prompt,
                max_tokens=100,  # Adjust as needed
                # ... other API parameters as needed ...
            )

            # (4) Extract and Process Response 
            extracted_strings = extract_benefits_strings(response.choices[0].text)
            benefits_strings.extend(extracted_strings)

        return format_output(benefits_strings, result_type)

    return process_document(load_data("./policy.pdf")) # Change the path as needed


[]

# GCP AI Platform 
Notebooks are pre-installed with the Rag V2 framework. This notebook demonstrates how to use the Rag V2 framework to fine-tune a Rag model on a custom dataset that is stored in a Google Cloud Storage bucket.
- The dataset should be in the form of a jsonl file with the following format:
```
{"text": "question", "meta": {"name": "document_name", "section": "document_section"}}
{"text": "answer", "meta": {"name": "document_name", "section": "document_section"}}
```

- The model is fine-tuned on the dataset using the `rag_v2` framework.
- The fine-tuned model is saved to a Google Cloud Storage bucket.


# Steps
1. Load the dataset from a Google Cloud Storage bucket.
2. Fine-tune the Rag model on the dataset.
3. Save the fine-tuned model to a Google Cloud Storage bucket.
4. Test the fine-tuned model.
5. Inference using the fine-tuned model + database of documents.

    a) Load the documents from a Google Cloud Storage bucket.

    b) Index the documents using embedding-model (LLM/NLP)

        i) Generate embeddings for the documents.

        ii) Index the embeddings

    c) Query the fine-tuned model with a question.

        i) Retrieve the top-k documents.

        ii) Retrieve the answer from the indexed documents.

    d) Retrieve the answer from the indexed documents.
    

## 1) GCP API
Test

In [2]:
import gspread

gc = gspread.service_account(filename='./gdrive/phrasal-ability-419201-d527372ace3b.json') 
# gc = gspread.service_account(filename='./gdrive/lunar-landing-389714-369d3f1b2a09.json')
sheets = gc.openall()
print([sheet.title for sheet in sheets]) 

['Master Database']


In [8]:
# Bedrock Command
cost_b = (1.5/1e6 + 2/1e6)*5000*1000
print(cost_b)
# cohere plus
cost_c = (0.5/1e6 + 1.5/1e6)*5000*1000
print(cost_c)

17.499999999999996
10.0


In [9]:
0.0001 *1000

0.1